Copyright (c) Microsoft Corporation. 
Licensed under the MIT license. 

In [1]:
from pyspark.sql import functions as F

In [2]:
account_name = '<<update your storage account name>>'
filesystem_name = 'sources'

path = 'abfss://%s@%s.dfs.core.windows.net' % (filesystem_name, account_name)

In [3]:
rawfile = 'Comparator SA Dataset.csv'

df_data = spark.read.csv(path+'/'+rawfile,header=True,sep=',')
display(df_data)
df_data.printSchema()

In [4]:
#temporary convert datatypes which should be done during data ingestion
def convert_string_to_floatWithRound(df_tmp):
    for c in ['ListPrice','StandardCost']:
        df_tmp =  (
            df_tmp
            .withColumn(c, F.col(c).cast("decimal"))
            .withColumn(c, F.round(F.col(c),2))
        )
    return df_tmp

def convert_string_to_int(df_tmp):
    for c in ['Size','Weight', 'DaysToManufacture']:
        df_tmp =  (
            df_tmp
            .withColumn(c, F.col(c).cast("int"))
        )
    return df_tmp


df_data = convert_string_to_int(df_data)
df_data = convert_string_to_floatWithRound(df_data)
display(df_data)

In [22]:
#Clean and Filter Data
df_name = df_data.dropna()
df_data = df_data.drop_duplicates()
df_data = df_data.na.drop()

display(df_data)


In [23]:
print(df_data.count())

In [24]:
# create unique ID
df_data = df_data.withColumn('ID', F.monotonically_increasing_id())
display(df_data)

In [25]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

In [26]:
df_data.write.mode('overwrite').saveAsTable('default.CleanComparator')